In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical

In [2]:
# Import data:
# 1. read the text file line by line;
# 2. format the data in DataFrame.

def read_data(path):
    data_list = []
    with open(path, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            d_str = line.split()
            d_tem = [float(d) for d in d_str]
            data_list.append(d_tem)
    data = pd.DataFrame(data_list)
    return data.T

# ready data for training:
# 1. sample_size=100: the most 100 recent updates
# 2. feature_num=40: 40 features per time stamp
# 3. target_num=5: relative changes for the next 1,2,3,5 and 10 events(5 in total)
def get_model_data(data, sample_size=100, feature_num=40, target_num=5):
    data = data.values
    shape = data.shape
    X = np.zeros((shape[0]-sample_size, sample_size, feature_num))
    Y = np.zeros(shape=(shape[0]-sample_size, target_num))
    for i in range(shape[0]-sample_size):
        X[i] = data[i:i+sample_size,0:feature_num]# take the first 40 columns as features
        Y[i] = data[i+sample_size-1,-target_num:]# take the last 5 columns as labels
    X = X.reshape(X.shape[0], sample_size, feature_num, 1)# add the 4th dimension: 1 channel
    
    # "Benchmark dataset for mid-price forecasting of limit order book data with machine learning"
    # labels 1: equal to or greater than 0.002
    # labels 2: -0.00199 to 0.00199
    # labels 3: smaller or equal to -0.002
    # Y=Y-1 relabels as 0,1,2
    Y = Y-1
    return X,Y
    
        

In [3]:
data_path = r'E:\JupyterFile\BenchmarkDatasets\BenchmarkDatasets\NoAuction\1.NoAuction_Zscore\NoAuction_Zscore_Training\Train_Dst_NoAuction_ZScore_CF_9.txt'
data = read_data(data_path)
train_X, train_Y = get_model_data(data)
train_Y = train_Y.astype(int)


In [5]:
train_X.shape

(362300, 100, 40, 1)

In [8]:
# the size of a single input is (100,40)
input_tensor = Input(shape=(100,40,1))

# convolutional filter is (1,2) with stride of (1,2)
layer_x = layers.Conv2D(16, (1,2), strides=(1,2))(input_tensor)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

layer_x = layers.Conv2D(16, (1,2), strides=(1,2))(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

layer_x = layers.Conv2D(16, (1,10))(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

# Inception Module
tower_1 = layers.Conv2D(32, (1,1), padding='same')(layer_x)
tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)
tower_1 = layers.Conv2D(32, (3,1), padding='same')(tower_1)
tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)

tower_2 = layers.Conv2D(32, (1,1), padding='same')(layer_x)
tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)
tower_2 = layers.Conv2D(32, (5,1), padding='same')(tower_2)
tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)  

tower_3 = layers.MaxPooling2D((3,1), padding='same', strides=(1,1))(layer_x)
tower_3 = layers.Conv2D(32, (1,1), padding='same')(tower_3)
tower_3 = layers.LeakyReLU(alpha=0.01)(tower_3)

layer_x = layers.concatenate([tower_1, tower_2, tower_3], axis=-1)

# concatenate features of tower_1, tower_2, tower_3
layer_x = layers.Reshape((100,96))(layer_x)

# 64 LSTM units
layer_x = LSTM(64)(layer_x)
# The last output layer uses a softmax activation function
output = layers.Dense(3, activation='softmax')(layer_x)
model = Model(input_tensor, output)

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 100, 40, 1)] 0                                            
__________________________________________________________________________________________________
conv2d_28 (Conv2D)              (None, 100, 20, 16)  48          input_3[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_28 (LeakyReLU)      (None, 100, 20, 16)  0           conv2d_28[0][0]                  
__________________________________________________________________________________________________
conv2d_29 (Conv2D)              (None, 100, 20, 16)  1040        leaky_re_lu_28[0][0]             
____________________________________________________________________________________________

In [ ]:
opt = tf.keras.optimizers.Adam(lr=0.01, epsilon=1)# learning rate and epsilon are the same as paper DeepLOB
y = to_categorical(train_Y[:,0])# y is the next event's mid price (k=1)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(train_X, y, epochs=100, batch_size=32)

W0819 21:54:52.619683  8128 deprecation.py:323] From E:\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
362300/362300 [==============================] - 1837s 5ms/sample - loss: 0.9045 - acc: 0.6388
Epoch 2/100
362300/362300 [==============================] - 1828s 5ms/sample - loss: 0.8130 - acc: 0.6598
Epoch 3/100
270112/362300 [=====================>........] - ETA: 7:46 - loss: 0.7450 - acc: 0.6844

In [7]:
test_data.to_csv('FI2010_test.csv')

In [6]:
test_path = r'E:\JupyterFile\BenchmarkDatasets\BenchmarkDatasets\NoAuction\1.NoAuction_Zscore\NoAuction_Zscore_Testing\Test_Dst_NoAuction_ZScore_CF_9.txt'
test_data = read_data(test_path)

In [ ]:
test_data = read_data(test_path)
test_X, test_Y = get_model_data(test_data)
test_Y = test_Y.astype(int)
test_y = to_categorical(test_Y[:,0])

model.evaluate(test_X, test_Y)